In [26]:
import pandas as pd


# Save merged_markdown_result to a markdown file
import os
from datetime import datetime, timedelta

ds = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")

# Create a directory for the markdown files if it doesn't exist
output_dir = f"output_qiyu_voice{ds}"
os.makedirs(output_dir, exist_ok=True)

import json


def parse_json_string(json_str):
    try:
        # Remove the prefix '```json' and suffix '```' if present
        cleaned_str = json_str.strip()
        if cleaned_str.startswith("```json") and cleaned_str.endswith("```"):
            cleaned_str = cleaned_str[7:-3].strip()
        elif cleaned_str.startswith("```") and cleaned_str.endswith("```"):
            cleaned_str = cleaned_str[3:-3].strip()

        # Parse the cleaned JSON string into a dictionary
        parsed_data = json.loads(cleaned_str)
        return parsed_data
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {"error": str(e), "original_input": json_str}

In [ ]:
# Import the base64 encoding library.
import base64, os, time
import logging
import httpx

# Configure the logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

proxy_object = {"http://": "http://127.0.0.1:7890", "https://": "http://127.0.0.1:7890"}

from openai import AzureOpenAI, OpenAI

my_gpt4o = OpenAI(base_url="https://api.gptsapi.net/v1")


def call_open_ai_directly(
    text_input,
    system_prompt=None,
    is_json=False,
    max_tokens=4096,
    model_to_use="gpt-4o-2024-08-06",
) -> (str, bool):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": text_input})

    try:
        completion = my_gpt4o.chat.completions.create(
            model=model_to_use,  # Assuming GPT-4 is available, adjust if needed
            messages=messages,
            max_tokens=max_tokens,
            temperature=0.1,
            response_format={"type": "json_object"} if is_json else {"type": "text"},
        )
        response = completion.choices[0].message.content
        logging.info(f"Usage:{completion.usage}")
        if response is None:
            logging.info(f"OpenAI API returned an exception: {completion}")
            return "OpenAI API returned an empty response", False
        return response, True
    except Exception as e:
        logging.error(f"Error calling OpenAI API: {str(e)}")
        return f"Error: {str(e)}", False


client_gpt4o = AzureOpenAI(
    api_version="2024-03-01-preview",
    azure_endpoint="https://xm-ai-us2.openai.azure.com",
    api_key=os.getenv("AZURE_GPT4O_API_KEY", ""),
    http_client=httpx.Client(proxies=proxy_object),
)

client_gpt4o_mini = AzureOpenAI(
    api_version="2024-03-01-preview",
    azure_endpoint="https://xm-ai-us.openai.azure.com",
    api_key=os.getenv("AZURE_GPT4O_MINI_API_KEY", ""),
)


def call_azure_openai(
    messages=[], retrying=1, is_gpt4o=False, json=True, max_tokens=4096
) -> (str, bool):
    if retrying < 0:
        return "超过了最大重试次数", False
    completion = None
    ## gpt3.5:  gpt-35-turbo-16k,
    ## got4o:   gpt-4o
    ## got4o-mini:   gpt-4o-mini
    model = "gpt-4o-mini"
    client_to_use = client_gpt4o_mini
    if is_gpt4o:
        logging.info(f"using GPT-4o...:{messages}")
        model = "gpt-4o"
        client_to_use = client_gpt4o
    try:
        completion = client_to_use.chat.completions.create(
            model=model,
            temperature=0.1,
            max_tokens=max_tokens,
            messages=messages,
            response_format={"type": "json_object"} if json else {"type": "text"},
        )
        response = completion.choices[0].message.content
        if (
            len(completion.choices) <= 0
            or f"{completion.choices[0].finish_reason}" == "content_filter"
        ):
            return f"azure过滤了本次请求:{completion.choices[0].to_dict()}", False
        if response is None:
            logging.info(f"azure API返回了异常:{completion.to_dict()}")
            time.sleep(10)
            return call_azure_openai(
                messages=messages,
                retrying=retrying - 1,
                is_gpt4o=is_gpt4o,
            )
        logging.info(f"total usage:{completion.usage}")
        return response, True
    except Exception as e:
        logging.info(
            f"请求azure接口报错了:{e}\n messages:{messages}, completion:{completion}"
        )
        if retrying <= 0 or "Error code: 400" in f"{e}":
            return f"{e}", False
        logging.info(f"重试中...{retrying}, messages:{messages}")
        return call_azure_openai(
            messages=messages,
            retrying=retrying - 1,
            is_gpt4o=is_gpt4o,
        )

In [ ]:
import sys, os

# Expand the `~` to the full path and append it to `sys.path`
full_path = os.path.expanduser("~/Documents/github/aliyun-devops")
sys.path.append(full_path)

from odps_client import get_odps_sql_result_as_df
from datetime import datetime, timedelta

staffname_list = ["白津源", "宋懿航", "李梦婷", "陈汉文"]
staffname_list = [f"'{staffname}'" for staffname in staffname_list]
staffname_list = ",".join(staffname_list)

sql = f"""
SELECT  a.*
        ,CAST(b.cust_id AS BIGINT) cust_id
        ,b.cust_name 商户名
        ,"电话拜访" AS 拜访目的
        ,"电话拜访" AS 拜访类型
        ,CASE   WHEN d.last_order_time IS NULL THEN '从未下单'
                ELSE '已下单'
        END AS 是否下过单
        ,DATEDIFF(GETDATE(),d.last_order_time,'dd') AS 距离上次下单天数
        ,od.历史下单数
        ,od.历史总下单金额
        ,DATEDIFF(GETDATE(),d.register_time,'dd') AS 注册天数
        ,c.m1_name AS M1负责人
        ,c.m2_name AS M2负责人
        ,c.m3_name AS M3负责人
        ,c.zone_name AS 销售区域
FROM    summerfarm_ds.crm_qiyu_call_feishu_result_raw_di a
LEFT JOIN summerfarm_tech.dim_cust_df b
ON      a.`to` = b.cust_phone
AND     b.ds = MAX_PT('summerfarm_tech.dim_cust_df')
LEFT JOIN summerfarm_tech.dim_bd_df c
ON      c.bd_name = a.staffname
AND     c.ds = MAX_PT('summerfarm_tech.dim_bd_df')
LEFT JOIN summerfarm_tech.ods_merchant_df d
ON      d.m_id = b.cust_id
AND     d.ds = MAX_PT('summerfarm_tech.ods_merchant_df')
LEFT JOIN   (
                SELECT  m_id
                        ,SUM(total_price) 历史总下单金额
                        ,COUNT(DISTINCT CASE    WHEN od.status IN (2,3,6) THEN od.order_no END) AS 历史下单数
                FROM    summerfarm_tech.ods_orders_df od
                WHERE   ds = MAX_PT('summerfarm_tech.ods_orders_df')
                GROUP BY m_id
            ) od
ON      od.m_id = b.cust_id
WHERE   a.ds = '{ds}'
and a.staffname in ({staffname_list})
;
"""
print(f"sql:{sql}")
bd_follow_up_record_df = get_odps_sql_result_as_df(sql=sql)
bd_follow_up_record_df.drop_duplicates(
    subset=["sessionid"], inplace=True
)

In [ ]:
# Display all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

bd_follow_up_record_df['communication_time_in_seconds']=bd_follow_up_record_df['communication_time_in_seconds'].astype(int)
bd_follow_up_record_df.drop_duplicates(subset=["sessionid"], inplace=True)

In [ ]:
bd_follow_up_record_df[['sessionid','feishu_file_recognize_result']].head(10)

In [ ]:
# 下载wav文件到本地，用于whisper模型识别

from concurrent.futures import ThreadPoolExecutor
import concurrent
import requests
import os


def download_wav_file(url, sessionid, communication_time_in_seconds=0):
    logging.info(f"下载语音文件:{url}, session:{sessionid}, 通话时长:{communication_time_in_seconds}s")
    file_name = os.path.basename(url)
    response = requests.get(url)
    local_file = f"{output_dir}/{sessionid}_{file_name}"
    if os.path.exists(local_file):
        logging.info(f"The file {local_file} already exists.")
        return

    # Check if the request was successful
    if response.status_code == 200:
        # Open a file in binary mode to write the content
        with open(local_file, "wb") as f:
            f.write(response.content)
        logging.info(f"File {file_name} downloaded successfully.")
    else:
        logging.info("Failed to download the file.")


# with ThreadPoolExecutor(max_workers=20) as executor:
#     futures = [
#         executor.submit(download_wav_file, row["recordurl"], row["sessionid"], row['communication_time_in_seconds'])
#         for index, row in bd_follow_up_record_df.iterrows()
#     ]
#     concurrent.futures.wait(futures)

## 调用whisper接口（Groq）

In [ ]:
def call_whisper_with_wav_file(row):
    logging.info(
        f"请求Groq进行语音识别, 文件:{row['recordurl']}, session:{row['sessionid']}, 通话时长:{row['communication_time_in_seconds']}s"
    )
    if int(row["communication_time_in_seconds"]) <= 10:
        error_text = (
            f"通话时长只有{row['communication_time_in_seconds']}s, 无需进行语音识别"
        )
        return {
            "text": error_text,
            "segments": error_text,
        }
    file_name = os.path.basename(row["recordurl"])
    local_file = f"{output_dir}/{row['sessionid']}_{file_name}"
    if not os.path.exists(local_file):
        logging.info(f"The file {local_file} does not exists.")
        return f"File not exists:{local_file}"

    return call_whisper_with_local_file(local_file)


import requests, time

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

groq_api_url = "https://api.groq.com/openai/v1/audio/transcriptions"

data = {
    "model": "whisper-large-v3",
    "language": "zh",
    "response_format": "verbose_json",
}


def call_whisper_with_local_file(
    filename="/Users/tangpeng/Downloads/白津源_3d341fed6a4637ad614830dc9d1a6b97.wav",
    is_retrying=False,
):
    with open(filename, "rb") as audio_file:
        files = {"file": (audio_file.name, audio_file)}

        headers = {
            "Authorization": f"Bearer {GROQ_API_KEY}",
        }

        response = requests.post(
            groq_api_url,
            headers=headers,
            data=data,
            files=files,
            proxies={"http": "http://127.0.0.1:7890", "https": "http://127.0.0.1:7890"},
        )

        if response.status_code == 200:
            transcription = response.json()
            print(f"Transcription:{transcription}")  # Example access
            return {
                "text": "\n".join(
                    [segment["text"] for segment in transcription["segments"]]
                ),
                "segments": [
                    f'{segment["start"]}s-{segment["end"]}s: {segment["text"]}'
                    for segment in transcription["segments"]
                ],
            }
        else:
            if not is_retrying and 400 <= response.status_code < 500:
                print(f"HTTP {response.status_code} error. Retrying in 45 seconds...")
                time.sleep(int(os.getenv("GROQ_API_RETRY_INTERVAL", "45")))
                return call_whisper_with_local_file(filename, is_retrying=True)
            else:
                print(f"is_retrying:{is_retrying}, Error:", response.status_code, response.text)
                return None


import concurrent.futures
from functools import partial


def process_row(row):
    result = call_whisper_with_wav_file(row)
    return pd.Series(result)


# Determine the number of threads to use (you can adjust this based on your system)
num_threads = min(32, os.cpu_count())

# Create a partial function with the apply method
process_func = partial(process_row)

# Use ThreadPoolExecutor for multi-threading
# with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
#     results = list(
#         executor.map(
#             process_func, [row for _, row in bd_follow_up_record_df.iterrows()]
#         )
#     )

# Assign the results back to the DataFrame
# bd_follow_up_record_df[["whisper_recongnize_text", "whisper_recongnize_segments"]] = (
#     pd.DataFrame(results)
# )

## 调用Open AI接口请求对话还原

In [19]:
json_output_prompt="""请仔细阅读销售员与客户之间的对话，并基于对话内容用 JSON 格式回答以下问题：

1. **客户是否与销售员进行了互动？**  
    - 请基于对话中的互动频率和深度回答，例如：客户是否回应了销售员的问题或建议。
   
2. **销售员推荐了哪些具体活动？**  
    - 请列出销售员提到的任何促销活动或优惠。

3. **销售员推荐了哪些具体商品？**  
    - 请列出销售员推荐的商品名称，如果销售员没有推荐商品，请回答“无”。

4. **客户的主要采购渠道？**  
    - 请根据客户提到的采购渠道（例如，淘宝、批发市场、拼多多等）回答。

5. **客户对公司的产品有何看法？（请列明具体产品名称）**  
    - 提取客户对公司产品的评价，并标明客户具体提到的产品名称。

6. **客户对公司的配送服务有何看法？**  
    - 请列出客户是否提到并评价公司的配送服务，如果没有提到，请回答“无”。

7. **客户对公司的总体评价？**  
    - 根据对话情感，判断客户对公司整体的评价类型（例如：正向/负向/中立/无法判断）。

8. **客户不愿意下单的原因？**  
    - 提取客户表达的不下单的原因，如果客户没有明确表达原因，请回答“无”。
    - 如果客户明确提到不开店了，或者倒闭了等等，也可以算做是不愿意下单的原因。

9. **销售员解决了客户哪些问题？**  
    - 列出销售员对客户问题的回答或澄清，说明哪些问题得到了明确解决。

10. **拜访记录完整性评分？（0-100 分，100 分为非常完整，0 分为非常不完整）**  
    - 根据对话的全面性、销售员的推荐情况、客户的互动等为拜访记录打分。

11. **对话总结**
    - 侧重客户的反馈内容，写一段50字以内的总结

---

**注意**:
- 商品名称如“安佳”、“铁塔”等属于具体商品，而非活动名称。活动名称通常会包含“专享”、“清仓”、“特价”、“活动”等词语。
- 回答仅基于对话内容，如果对话中找不到问题答案，请回答“无”。
- 请按以下格式返回：
{
  "客户是否有跟销售员互动": "[]",
  "销售向客户推荐了哪些具体的活动": "[]",
  "销售向客户推荐了哪些具体的商品": "[]",
  "客户的主要采买渠道": "[]",
  "客户对公司的产品有什么看法": "[]",
  "客户对公司的配送服务有什么看法": "[]",
  "客户对公司的评价是怎样的": "[]",
  "客户不愿意下单的原因": "[]",
  "销售员解决了客户哪些问题？": "[]",
  "拜访记录完整性打分": 75,
  "一句话总结以上对话": "[]"
}
"""

In [ ]:
import json
from typing import Dict, Any, Tuple


def process_conversation(row: pd.Series) -> Tuple[str, Dict[str, Any]]:
    # Step 1: Restore the conversation using OpenAI
    conversation_prompt = f"""
根据用户给出转录内容，请严格基于提供的文本信息，还原销售员和客户之间的对话。请勿进行任何额外的推测或创造，所有对话内容必须严格源于转录内容。

请按以下格式还原对话：

销售员: [销售员的对话]
客户: [客户的对话]

请注意：

1. 只根据转录内容还原对话，不能添加任何未提供的信息。
2. 如果转录内容不完整或含糊，请仅展示可识别的部分，不要自行补全。
    """

    restored_conversation = call_open_ai_directly(
        text_input=row["feishu_file_recognize_result"],
        system_prompt=conversation_prompt,
    )[0]

    logging.info(f"restored_conversation:{restored_conversation}")

    analysis_result = call_open_ai_directly(
        text_input=restored_conversation, system_prompt=json_output_prompt
    )[0]

    logging.info(
        f"restored_conversation:{restored_conversation}\nanalysis_result:{analysis_result}"
    )

    return restored_conversation, parse_json_string(analysis_result)


# Apply the function to each row of the DataFrame
bd_follow_up_record_df[["还原对话", "AI分析"]] = bd_follow_up_record_df.apply(
    lambda row: pd.Series(process_conversation(row)), axis=1
)

In [25]:
input = """客户: 今天我说什么？我怎么听不清你说话啊？

销售员: 我，您刚可以听听吗？

客户: 啊啊，对，因为咱们老表的话，我这边是仙牧农场的。

销售员: 啊，对，因为咱们最近商城的话是有一些活动嘛，所以说也是把这个消息给您同步一下。

客户: 为什么？

销售员: 就是我们最近的话是有牛奶，还有奶油这块的一个活动，带奶油这块安家还有铁塔这一块的一些活动。然后包括说咱们这个目前水果这块用的到嘛？用的多不多？

客户: 这边我们，我们只会用一些你们的那些水果，其他的都是在别的地方买的啊？

销售员: 其他都在别的地方买，是吧？

客户: 啊？

销售员: 那最近水果这一块感觉品质这块怎么样？

客户: 诶？我最近好像没买过诶。你的量比较大，我们更多的都在京东上版。

销售员: 哦，是京东是隔两天到，是吧？还是说是当天给您送到那种京东吧？

客户: 京东我们只买牛奶，京东，我买牛奶其他的一些东西，然后我们在叮咚上买水果，买的更多，你们的是买一小部分啊？

销售员: 对，你们的品质不是特别好，还是分一会分成一级国、二级国、三级国，有时候老是买错啊啊，而且东西亮弹太大。

客户: 明白，明白，因为我们这边的话其实主要也是对于针对一些门店嘛，如果说您觉得这边量大的话，您可以到时候看一下，或者是少买一点，对，按那个最小的量买，咱们就是你们的价格也不是很优惠啊？

销售员: 价格这一块您之前看的是哪一块的一个价格？水果这边吗？

客户: 我们，我们大部分水果跟叮咚每天采购，叮咚就比你们便宜，要便宜一些的。

销售员: 嗯，您是对比，是同一个，同一个，比如说品质的一个芒果，是吧？比如说凯特芒这个，或者说是披帽这块，我们，我们更多的用的是香蕉和火龙果，火龙果基本上会在你这买的多，但是香蕉就你的价格也不是很便宜，而且量太大。

客户: 明白明白。

销售员: 好的，您或者回头的话，最近这边火龙果的话就是越南这一块的，可能结合下来就五块多一斤，你可以看一下这边它有五斤一包的，您这边应该两三天应该可以用完吧？

客户: 5天之前买，因为他有那个好像说不是特别多，明白明白。

销售员: 好的，行，因为咱们最近的话就是水果这块，你也可以看一下这边那价格的话可能比上个月稍微划算一点，然后如果说您十几斤用不完的话，可以买三四斤，5斤左右左右，这个应该可以用的。

客户: 哇，这个呀，对，你可以看一下越南红星，越南的这款等你的价格也不优惠啊。

销售员: 明白，那你们这种太麻烦，简单讲，我今天买，明天才能送到，我今天买了一小会就送来了。

客户: 明白，明白，可能就是您比较需要的，就是比如说当天买，当天送的那种，是吧？

销售员: 对啊，我，我不买东西，自己买10斤吗？用完马上就可以送过来的，但是从你这边要买，要买一箱要二十几斤，三十斤的吧？咱三十几斤我们用不完就放，时间长就坏了，但是。

客户: 你要是不够了，你再左右，没有当天能送，所以基本上用你们家东西很少，我们牛奶也都是在京东上统一买的。

销售员: 嗯嗯，明白明白。好的，牛奶这一块的话是用哪个牌子的？用糯吗？

客户: 安家的话，安家的，是吧？

销售员: 对，因为我们这边的话也有我们这一个牌子，有个 PPT 奶，还有英诺这一块，然后也卖得不错。如果是玩家的啊，就是专门挑的一个牌子用的，是吧？

客户: 对哦。

销售员: 好嘞好嘞，那牛奶用的量大，都是那个的牛奶，您可以看一下，因为我们有两款奶还是不错的，这边就买的比较多，这个的它咖啡店也会有，经常买一块。

客户: 嗯，帮我操控关注一下吧。

销售员: 好的，好嘞，行不客气？

客户: 行，那这边先不打扰您，哎，嗯，好，拜拜。"""


output = call_open_ai_directly(text_input=input, system_prompt=json_output_prompt)[0]
print(parse_json_string(output))

2024-09-14 10:54:40 - INFO - HTTP Request: POST https://api.gptsapi.net/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-14 10:54:40 - INFO - Usage:CompletionUsage(completion_tokens=231, prompt_tokens=1825, total_tokens=2056, completion_tokens_details={'reasoning_tokens': 0})


{'客户是否有跟销售员互动': '是', '销售向客户推荐了哪些具体的活动': ['牛奶和奶油活动'], '销售向客户推荐了哪些具体的商品': ['安佳', '铁塔', '越南红星火龙果', 'PPT奶', '英诺'], '客户的主要采买渠道': ['京东', '叮咚'], '客户对公司的产品有什么看法': ['水果品质不佳', '价格不优惠'], '客户对公司的配送服务有什么看法': '无', '客户对公司的评价是怎样的': '负向', '客户不愿意下单的原因': ['价格不优惠', '配送不及时', '购买量要求过大'], '销售员解决了客户哪些问题？': '无', '拜访记录完整性打分': 75, '一句话总结以上对话': '客户对公司产品和服务不满，主要通过京东和叮咚采购。'}


In [27]:


# Example usage
json_str = '''```json
{
  "客户是否有跟销售员互动": "是，交流了10句话",
  "销售向客户推荐了哪些具体的活动": "无",
  "销售向客户推荐了哪些具体的商品": "芒果，葡萄",
  "客户的主要采买渠道": "批发市场，淘宝，拼多多",
  "客户对公司的产品有什么看法？": "价格较高，质量一般",
  "客户对公司的配送服务有什么看法？": "无",
  "客户对公司的评价是怎样的？": "负向",
  "客户不愿意下单的原因": "价格过高",
  "销售员解决了客户哪些问题？": "无",
  "拜访记录完整性打分": 75,
  "一句话总结以上对话": "客户反馈我们的葡萄很贵，比拼多多等贵很多"
}
```'''

parsed_json = parse_json_string(json_str)
print(parsed_json)

{'客户是否有跟销售员互动': '是，交流了10句话', '销售向客户推荐了哪些具体的活动': '无', '销售向客户推荐了哪些具体的商品': '芒果，葡萄', '客户的主要采买渠道': '批发市场，淘宝，拼多多', '客户对公司的产品有什么看法？': '价格较高，质量一般', '客户对公司的配送服务有什么看法？': '无', '客户对公司的评价是怎样的？': '负向', '客户不愿意下单的原因': '价格过高', '销售员解决了客户哪些问题？': '无', '拜访记录完整性打分': 75, '一句话总结以上对话': '客户反馈我们的葡萄很贵，比拼多多等贵很多'}


In [ ]:
import json

bd_follow_up_record_df = bd_follow_up_record_df.dropna(subset=["商户名"])
bd_follow_up_record_df.to_csv(f"{output_dir}/bd_follow_up_record_df.csv", index=False)

bd_follow_up_record_df.rename(
    columns={
        "feishu_file_recognize_result": "拜访内容_飞书",
        "whisper_recongnize_text": "拜访内容_whisper",
        "whisper_recongnize_segments": "拜访内容_segments",
        "communication_time_in_seconds": "通话时长s",
        "staffname": "拜访人",
    },
    inplace=True,
    errors="ignore",
)


whisper_segments_csv = bd_follow_up_record_df[
    ["商户名", "拜访人", "是否下过单", "距离上次下单天数", "拜访内容_segments"]
]
whisper_segments_csv["拜访内容_segments"] = whisper_segments_csv[
    "拜访内容_segments"
].apply(lambda segments: ", ".join(segments) if isinstance(segments, list) else "")
whisper_segments_csv.to_csv(
    f"{output_dir}/拜访内容_飞书_whisper_segments.csv", index=False
)

In [ ]:
# Extract '拜访记录完整性打分' from 'AI分析' column
bd_follow_up_record_df['拜访记录完整性打分'] = bd_follow_up_record_df['AI分析'].apply(
    lambda x: json.loads(x).get('拜访记录完整性打分', '未知')
)

# Convert to numeric, replacing '未知' with NaN
bd_follow_up_record_df['拜访记录完整性打分'] = pd.to_numeric(bd_follow_up_record_df['拜访记录完整性打分'], errors='coerce')


In [ ]:
# Create a new column 'AI总结' by calling the Azure OpenAI API for each row
bd_follow_up_record_df["AI总结"] = bd_follow_up_record_df.apply(
    lambda row: (
        "拜访记录不够完整，无需AI总结"
        if row["拜访记录完整性打分"] < 60
        else call_azure_openai(
            is_gpt4o=False,
            json=False,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"""从以下销售员和客户之间的通话录音中，你发现了哪几个最值得公司管理者关注的问题？
请你结合客户的基本情况，以及销售员拜访的记录，列举出明确的值得管理层引起注意的内容，并且标注出处(发生在录音内容的哪一个时段)\n\n
你不需要给出建议，只需要列出值得管理层引起注意的内容，并且标注出处\n\n
客户距离上次下单天数:{row['距离上次下单天数']}, 客户历史总下单金额:¥{row['历史总下单金额']}, 通话时长:{row['通话时长s']}s\n拜访内容:\n{row['拜访内容_segments']}""",
                        }
                    ],
                },
            ],
        )[0]
    ),
    axis=1,
)

In [ ]:
bd_follow_up_record_df.sort_values(by="拜访记录完整性打分", ascending=False, inplace=True)
# bd_follow_up_record_df[["AI总结","AI分析","拜访记录完整性打分","拜访内容_whisper"]].head(20)

In [ ]:
from openai import OpenAI

client = OpenAI(base_url="https://api.gptsapi.net/v1")

audio_file = open(f"{output_dir}/9225272546_6fd715f05411116867da5a2c860da7ce.wav", "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    response_format="verbose_json",
    timestamp_granularities=["segment"],
)

print(transcript)

In [ ]:
import json

keys = []


def extract_ai_result(ai_result, key):
    return json.loads(ai_result).get(key, "未知")


for city in bd_follow_up_record_df["拜访人"].unique():
    logging.info(f"开始处理:{city}的拜访记录")
    sale_man_df = bd_follow_up_record_df[
        bd_follow_up_record_df["拜访人"] == city
    ].copy()

    # Create a valid filename by replacing any characters that might be problematic in filenames
    safe_city_name = "".join(c if c.isalnum() or c in ("-", "_") else "_" for c in city)

    # Save the city's records to a CSV file
    filename = f"./{output_dir}/{safe_city_name}_{ds}_拜访记录.csv"
    sale_man_df[
        [
            "拜访人",
            "m1负责人",
            "商户名",
            "距离上次下单天数",
            "拜访记录完整性打分",
            "历史下单数",
            "历史总下单金额",
            "拜访内容_whisper",
            "AI分析",
            "recordurl",
        ]
    ].to_csv(filename, index=False, encoding="utf-8-sig")

    print(f"Saved {len(sale_man_df)} records for {city} to {filename}")

    for index, row in sale_man_df.iterrows():
        ai_result = json.loads(row["AI分析"])
        if not keys:
            keys = list(ai_result.keys())
            logging.info(f"keys: {keys}")
            break

    for key in keys:
        sale_man_df[key] = sale_man_df["AI分析"].apply(
            lambda x: extract_ai_result(x, key)
        )

    display_keys = [
        "销售区域",
        "拜访人",
        "m1负责人",
        "商户名",
        "距离上次下单天数",
        "拜访记录完整性打分",
        "历史下单数",
        "历史总下单金额",
        "拜访内容_whisper",
        "AI分析",
        "AI总结",
    ]
    display_keys.extend(keys)
    sale_man_df[display_keys].to_csv(
        f"./{output_dir}/{safe_city_name}_{ds}_拜访记录_AI分析_展开.csv", index=False
    )

    ai_csv_analytics_keys = [
        "销售区域",
        "拜访人",
        "m1负责人",
        "商户名",
        "距离上次下单天数",
        "拜访记录完整性打分",
        "历史下单数",
        "历史总下单金额",
    ]
    ai_csv_analytics_keys.extend(keys)
    csv_string = sale_man_df[ai_csv_analytics_keys].to_csv(index=False)

    print(f"{city}, \ncsv_string:{csv_string}")
    call_ai_api_to_get_insigns(csv_string=csv_string, city=safe_city_name)